<a href="https://colab.research.google.com/github/ccpptr/ETL-Receita-Federal/blob/main/Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Considerações:


1.   Inicialmente, esbarrei em um erro que nunca havia presenciado antes, no que diz respeito à abertura dos arquivos usando o comando ***pd.read_csv*()*** pois o panda estava esperando um arquivo ***utf8*** enquanto seria necessário um encoding  ***iso8859-1*** ou senão o atributo ***encoding_errors='ignore'***

2.   Logo após tive problemas ao abrir a base de dados em meu computador pessoal, em especial o arquivo de Estabelecimentos que possuía mais de 2gb. Portanto tive que partir para uma solução usando o Colab, e ainda assim precisei tomar cuidado com a ordem de execução das células para que não estourasse a RAM disponível.

3. Ressalto a importância das soluções vetorizadas oferecidas pelo Pandas como os métodos ***map()/apply()*** e ***merge()/join()/concat()*** em conjunto com os dicionários como uma forma de substituir e otimizar os loops e condicionais tradicionais como ***for*** e ***if***, uma vez que o tempo de execução decai absurdamente e torna o código mais fluido e legível 




Montar o Drive no Colab, ignorar se estiver usando o jupyter notebook no local host

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Alterar o diretório onde estão os arquivos da base de dados

In [ ]:
import pandas as pd
diretorio="/content/drive/MyDrive/Colab Notebooks/"


Abertura das bases de dados e nomeação das colunas


In [ ]:
#Abertura dos arquivos das bases de dados
estabele= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/K3241.K03200Y0.D20514.ESTABELE', sep=';', encoding='iso8859-1')
empresa = pd.read_csv(diretorio+'K3241.K03200Y0.D20514.EMPRECSV', sep=';',encoding='iso8859-1')
socio = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/K3241.K03200Y0.D20514.SOCIOCSV', sep=';', encoding='iso8859-1')

#NOMEAR AS COLUNAS
socio.columns = ["CNPJ BÁSICO", "IDENTIFICADOR DE SÓCIO", "NOME DO SÓCIO OU RAZÃO SOCIAL", "CNPJ/CPF DO SÓCIO",
              "QUALIFICAÇÃO DO SÓCIO", "DATA DE ENTRADA SOCIEDADE", "PAÍS SÓCIO", "REPRESENTANTE LEGAL", 
              "NOME DO REPRESENTATE", "QUALIFICAÇÃO DO REPRESENTANTE LEGAL", "FAIXA ETÁRIA"]
estabele.columns = ["CNPJ BÁSICO", "CNPJ ORDEM", "CNPJ DV", "ID MATRIZ/FILIAL", "NOME FANTASIA", 
                 "SITUAÇÃO CADASTRAL","DATA SITUAÇÃO CADASTRAL", "MOTIVO SITUAÇÃO CADASTRAL", "NOME DA CIDADE NO EXTERIOR",
                 "PAÍS", "DATA DE INICIO ATIVIDADE", "CNAE FISCAL PRINCIPAL", "CNAE FISCAL SECUNDÁRIA",
                 "TIPO DE LOUGRADOURO", "LOUGRADOURO", "NUMERO", "COMPLEMENTO", "BAIRRO", "CEP", "UF",
                 "MUNICÍPIO", "DDD 1","TELEFONE 1", "DDD 2", "TELEFONE 2", "DDD DO FAX", "FAX",
                 "CORREIO ELETRÔNICO", "SITUAÇÃO ESPECIAL", "DATA DA SITUAÇÃO"]
empresa.columns = ["CNPJ BÁSICO", "RAZÃO SOCIAL", "NATUREZA JURÍDICA", "QUALIFICAÇÃO DO RESPONSÁVEL",
                 "CAPITAL SOCIAL", "PORTE DA EMPRESA", "ENTE FEDERATIVO RESPONSÁVEL"]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Merges das bases de dados de forma a combinar somente as linhas que possuem o "CNPJ BÁSICO" em comum, primeiro para as empresas e depois para os sócios, usando a base de dados dos estabelecimentos como o DataFrame principal.

In [ ]:
#socio=socio.drop(columns = ['PAÍS'])
estabele = pd.merge(estabele, empresa, how='inner', on ="CNPJ BÁSICO")
estabele = pd.merge(estabele, socio, how='inner', on ="CNPJ BÁSICO")

Substituição dos Códigos pelos valores das tabelas




In [ ]:
################################
############TABELAS#############
################################
tabela_cnae = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F.K03200$Z.D20514.CNAECSV', sep=';', encoding='iso8859-1')
tabela_motivo = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F.K03200$Z.D20514.MOTICSV', sep=';', encoding='iso8859-1')
tabela_municipio = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F.K03200$Z.D20514.MUNICCSV', sep=';', encoding='iso8859-1')
tabela_pais = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F.K03200$Z.D20514.PAISCSV', sep=';', encoding='iso8859-1')
tabela_natjur= pd.read_csv(diretorio+'F.K03200$Z.D20514.NATJUCSV', sep=';', encoding='iso8859-1')
tabela_qualsocio= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/F.K03200$Z.D20514.QUALSCSV', sep=';', encoding='iso8859-1')

dictcnae = {tabela_cnae.iloc[i,0]:tabela_cnae.iloc[i,1]for i in range(len(tabela_cnae.index))}
estabele["CNAE FISCAL PRINCIPAL"]=estabele["CNAE FISCAL PRINCIPAL"].map(dictcnae)
estabele["CNAE FISCAL SECUNDÁRIA"]=estabele["CNAE FISCAL SECUNDÁRIA"].map(dictcnae)

dictmotivo = {tabela_motivo.iloc[i,0]:tabela_motivo.iloc[i,1]for i in range(len(tabela_motivo.index))}
estabele["MOTIVO SITUAÇÃO CADASTRAL"]=estabele["MOTIVO SITUAÇÃO CADASTRAL"].map(dictmotivo)

dictmunicipio = {tabela_municipio.iloc[i,0]:tabela_municipio.iloc[i,1]for i in range(len(tabela_municipio.index))}
estabele["MUNICÍPIO"]=estabele["MUNICÍPIO"].map(dictmunicipio)

dictpais = {tabela_pais.iloc[i,0]:tabela_pais.iloc[i,1]for i in range(len(tabela_pais.index))}
estabele["PAÍS"]=estabele["PAÍS"].map(dictpais)
estabele["PAÍS SÓCIO"]=estabele["PAÍS SÓCIO"].map(dictpais)

dictnatjur = {tabela_natjur.iloc[i,0]:tabela_natjur.iloc[i,1]for i in range(len(tabela_natjur.index))}
estabele["NATUREZA JURÍDICA"]=estabele["NATUREZA JURÍDICA"].map(dictnatjur)

dictqualsocio = {tabela_qualsocio.iloc[i,0]:tabela_qualsocio.iloc[i,1]for i in range(len(tabela_qualsocio.index))}
estabele["QUALIFICAÇÃO DO SÓCIO"]=estabele["QUALIFICAÇÃO DO SÓCIO"].map(dictqualsocio)


Dicionários auxiliares presentes no NOVO LAYOUT da RF

In [ ]:
#porte da empresa
dictporte = {"00":"Não Informado", "01":"Micro Empresa", "03":"Empresa de Pequeno Porte","05":"Demais"}
estabele["PORTE DA EMPRESA"]=estabele["PORTE DA EMPRESA"].map(dictporte)

#id matriz/filial
dictid = {1:"Matriz",2:"Filial"}
estabele["ID MATRIZ/FILIAL"]=estabele["ID MATRIZ/FILIAL"].map(dictid)

#situação cadastral
dictsitu = {"01":"Nula", "2":"Ativa", "3":"Suspensa", "4":"Inapta", "08":"Baixada"}
estabele["SITUAÇÃO CADASTRAL"]=estabele["SITUAÇÃO CADASTRAL"].map(dictsitu)

#id socio
dictidsocio = {1:"Pessoa Jurídica", 2:"Pessoa Física",3:"Estrangeiro"}
estabele["IDENTIFICADOR DE SÓCIO"]=estabele["IDENTIFICADOR DE SÓCIO"].map(dictidsocio)

Ordenar a base final pelo "CNPJ BÁSICO" de forma decrescente, remover as colunas vazias e imprimir as 1000 primeiras linhas num Output de Excel

In [ ]:
estabele=estabele.sort_values(estabele.columns[0], ascending=False)
estabele=estabele.head(999)
estabele=estabele.dropna(axis=1, how='all', thresh=None, subset=None, inplace=False)
estabele.to_excel("/content/drive/MyDrive/Colab Notebooks/base_final.xls", index=False)
estabele.to_excel("/content/drive/MyDrive/Colab Notebooks/base_final.xlsx", index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  after removing the cwd from sys.path.
